# API Profile

In [3]:
import base64
import requests, json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

from itsdangerous import URLSafeTimedSerializer
from IPython.display import display

In [4]:
TM_SECRET = '0cNOkuyerq9J6vEXics6nTwPnjBgFeGF1Pq6yVy9'
TM_APP_BASE_URL = 'http://127.0.0.1:5000'

In [5]:
def create_token(user_id):
    serializer = URLSafeTimedSerializer(TM_SECRET)
    token = serializer.dumps(user_id)
    encoded = base64.b64encode(bytes(token, 'utf-8')).decode('utf-8')

    return encoded

In [6]:
OSM_User_ID_number = 834883 
TOKEN = create_token(OSM_User_ID_number)

In [7]:
payloads_examples = {
    "organisations": [105],
    "campaign_id": 237,
    "license_id": 5,
    "project_id": 8410,
    "task_id": 186,
    "interest_id": 3,
    "organisation_id": 105,
    "message_id": 3,
    "category_id": 3,
    "team_id": 1,
    "username": "nahomy",
    "user_id": OSM_User_ID_number,
}

payload_post_patch_examples = {
    "/api/v2/interests/": {"name": "covid"},
    "/api/v2/interests/{interest_id}/": {"name": "Public Domain 2"},
    "/api/v2/projects/{project_id}/actions/set-interests/": {"interests": [3]},
    "/api/v2/licenses/": {
        "description": "licencia 2",
        "name": "licencia 2",
        "plainText": "licencia 2",
    },
    "/api/v2/organisations/": {
        "logo": "string",
        "managers": ["nahomy"],
        "name": "test",
        "url": "string",
    },
    "/api/v2/organisations/{organisation_id}/": {
        "logo": "string",
        "managers": ["nahomy"],
        "name": "test",
        "url": "string",
    },
    "/api/v2/projects/": {
        "cloneFromProjectId": 8410,
        "projectName": "Mapathon Bahia Blanca",
        "areaOfInterest": {
            "type": "FeatureCollection",
            "features": [
                {
                    "id": "65d4fe238d4356225e599c1b72e070d0",
                    "type": "Feature",
                    "properties": {},
                    "geometry": {
                        "coordinates": [
                            [
                                [-62.363229765845944, -38.70359480488574],
                                [-62.28349668643837, -38.61738800371326],
                                [-62.14505903208223, -38.716585238332854],
                                [-62.25283011743514, -38.78764893912241],
                                [-62.363229765845944, -38.70359480488574],
                            ]
                        ],
                        "type": "Polygon",
                    },
                }
            ],
        },
        "tasks": {
            "type": "FeatureCollection",
            "features": [
                {
                    "type": "Feature",
                    "properties": {"x": 167, "y": 196, "zoom": 9, "isSquare": True},
                    "geometry": {
                        "type": "MultiPolygon",
                        "coordinates": [
                            [
                                [
                                    [-62.578124997501575, -38.82259097486486],
                                    [-61.87499999752964, -38.82259097486486],
                                    [-61.87499999752964, -38.272688534680675],
                                    [-62.578124997501575, -38.272688534680675],
                                    [-62.578124997501575, -38.82259097486486],
                                ]
                            ]
                        ],
                    },
                }
            ],
        },
        "arbitraryTasks": False,
    },
    "/api/v2/projects/{project_id}/": {
        "allowedUsernames": ["string"],
        "campaign": "string",
        "changesetComment": "string",
        "countryTag": ["string"],
        "defaultLocale": "string",
        "dueDate": "Unknown Type: date",
        "entitiesToMap": "string",
        "imagery": "string",
        "josmPreset": "string",
        "licenseId": 5,
        "mapperLevel": "string",
        "mappingEditors": ["string"],
        "mappingTypes": ["string"],
        "mapping_permission": "string",
        "organisation": 105,
        "priorityAreas": [],
        "private": False,
        "projectInfoLocales": [],
        "projectPriority": "string",
        "projectStatus": "string",
        "taskCreationMode": 0,
        "validationEditors": ["string"],
        "validation_permission": "string",
    },
    "/api/v2/projects/{project_id}/actions/transfer-ownership/": {"username": "nahomy"},
}


In [8]:
def get_parameters(payloads):
    for payload in payloads:
        if payload in payloads_examples:
            payloads[payload] = payloads_examples[payload]
    return payloads

In [9]:
def get_parameters_post_patch(key, payloads):
    if key in payload_post_patch_examples:
        for value, content in payload_post_patch_examples[key].items():
            payloads[value] = payload_post_patch_examples[key][value]
    return payloads

In [10]:
def set_parameters(endpoint, headers, paths, params, payloads):
        
        for method, values in endpoint.items():
            if "parameters" in endpoint[method]:
                for param in endpoint[method]["parameters"]:
                    if param["in"] == "header":
                        
                        if param["name"] == "Authorization":
                            headers.update({param["name"]: f"Token {TOKEN}"})
                        else:
                            if "default" in param:
                                headers.update({param["name"]: param["default"]})

                    elif param["in"] == "path":
                        if "default" in param:
                            paths.update({param["name"]: param["default"]})                          

                    elif param["in"] == "query":
                        if "default" in param:
                            params.update({param["name"]: param["default"]})
                            
                    elif param["in"] == "body":
                        if "properties" in param["schema"]:
                            properties = param["schema"]["properties"]
                            for item, propertie in properties.items():
                                if "default" in propertie:
                                    payloads.update({item: propertie["default"]})
                                elif "example" in propertie:
                                    payloads.update({item: propertie["example"]})

In [11]:
def add_data_response(endpoint, method, url, resp):
    
    data["Model"].append(endpoint[method]["tags"][0])
    data["Method"].append(method)
    data["API_url"].append(url)
    data["Time"].append(resp.elapsed.total_seconds())
    data["Status_code"].append(resp.status_code)

In [12]:
def calculate_time_response(endpoints, data_time_response):

    for key, endpoint in endpoints.items():
        url = ""
        headers = {}
        paths = {}
        params = {}
        payloads = {}
                
        set_parameters(endpoint, headers, paths, params, payloads)

        if paths:
            url = key.format(**get_parameters(paths))
        else:
            url = key
            
        if "get" in endpoint:
            resp = requests.get(TM_APP_BASE_URL + url, headers=headers, params=params)
            add_data_response(endpoint,"get", url, resp)

        if "post" in endpoint:
            resp = requests.post(
                TM_APP_BASE_URL + url,
                headers=headers,
                params=params,
                json=get_parameters_post_patch(key, payloads),
            )
            add_data_response(endpoint,"post", url, resp)

        if "patch" in endpoint:
            resp = requests.patch(
                TM_APP_BASE_URL + url,
                headers=headers,
                params=params,
                json=get_parameters_post_patch(key, payloads),
            )
            add_data_response(endpoint,"patch", url, resp)

        if "delete" in endpoint:
            resp = requests.delete(
                TM_APP_BASE_URL + url,
                headers=headers,
                params=params,
                json=get_parameters_post_patch(key, payloads),
            )
            add_data_response(endpoint,"delete", url, resp)

In [13]:
data = {
    "Model":[],
    "Method":[],
    "API_url": [],
    "Time": [],
    "Status_code": []
}

def main():
    
    endpoints = json.loads(
        requests.get("http://127.0.0.1:5000/api/v2/system/docs/json/").content
    )
    calculate_time_response(endpoints["paths"], data)
    

if __name__ == '__main__':
    main()

In [14]:
def show_table_graph(model):
    grouped = data_frame.groupby(data_frame.Model)    
    group = grouped.get_group(model)
    layout = go.Layout(title=f'Response time: {model}')
    table = go.Figure(layout=layout, data=[go.Table(
        columnwidth = [80,80,300,80,80],
        header=dict(values=list(group.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[group.Model, group.Method, group.API_url,
                           group.Time, group.Status_code],
                   fill_color='lavender',
                   align='left',
                   height=40))
    ])
    table.show()
    chart = px.bar(group, x='Time', y='API_url',orientation='h', title=f'Response time: {model}')
    display(chart)

In [ ]:
data_frame = pd.DataFrame(data)
models = list(dict.fromkeys(data_frame["Model"]))

interact(show_table_graph, model=models)


interactive(children=(Dropdown(description='model', options=('campaigns', 'countries', 'interests', 'licenses'…

### Summary of the 10 endpoints that have a very slow response time

In [ ]:
data_top_slow = data_frame.nlargest(10, 'Time')

layout = go.Layout(title='Summary of the 10 endpoints that have a very slow response time')
table = go.Figure(layout=layout, data=[go.Table(
        columnwidth = [80,80,300,80,80],
        header=dict(values=list(data_top_slow.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[data_top_slow.Model, data_top_slow.Method, data_top_slow.API_url,
                           data_top_slow.Time, data_top_slow.Status_code],
                   fill_color='lavender',
                   align='left',
                   height=40))
    ])
table.show()

chart = px.bar(data_top_slow, x='API_url', y='Time', 
               orientation='v',
               color='Time')
chart.show()